# Defect Injection in SPICE Netlists
This Jupyter Notebook was created to explore injecting defects into SPICE netlists based in the IEEE P2427 framework.

It involves the following process:

1.   Devices and nodes are in the netlist are parsed.
2.   Defects are added into the netlist. These are modelled as large parameter changes, shorts or opens.
3.   The netlist is simulated and performance parameters are measured. If the performance parameters fall outside of the expected range, they are classified as failures.
4. Defect coverage is determined based in the percentage of defects detected by the tests.

In [ ]:
# Import modules
import re

## Example SPICE Netlist

This netlist is a testbench for testing a simple 5 transistor OpAmp.

In [ ]:
# Download SPICE netlist from github
spice_netlist_url = f"https://raw.githubusercontent.com/TimothyJNewman/open-source-chip-design-v1/refs/heads/main/ISSCC_2025_V1/netlist/test_analog.spice"
!wget {spice_netlist_url}
!mkdir generated_netlists

--2024-11-24 00:12:39--  https://raw.githubusercontent.com/TimothyJNewman/open-source-chip-design-v1/refs/heads/main/ISSCC_2025_V1/netlist/test_analog.spice
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5226 (5.1K) [text/plain]
Saving to: ‘test_analog.spice.6’

test_analog.spice.6 100%[===================>]   5.10K  --.-KB/s    in 0s      

2024-11-24 00:12:39 (51.7 MB/s) - ‘test_analog.spice.6’ saved [5226/5226]

mkdir: cannot create directory ‘generated_netlists’: File exists


In [ ]:
# Open SPICE netlist
with open("test_analog.spice", "r") as netlist_file:
  netlist_contents = netlist_file.read()

# Print file in terminal
  print(netlist_contents)

** sch_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/test_analog/test_analog.sch
**.subckt test_analog
V6 vin_p GND 0.9 dc 0.9 ac 1 pulse(0 1 100p 100p 100p 1u 2u)
V7 vin_n GND 0.9
V8 VDD GND 1.8
V9 VSS GND 0
I0 VSS i_bias 10u
x1 VDD vout vin_p vin_n VSS i_bias en OpAmp
V1 en GND 1.8
C1 vout GND 1p m=1
**** begin user architecture code
.lib /usr/local/share/pdk/sky130B/libs.tech/combined/sky130.lib.spice tt
**** end user architecture code
**.ends

* expanding   symbol:  Benchmark_Circuits/OpAmp_Defect/OpAmp.sym # of pins=7
** sym_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/OpAmp_Defect/OpAmp.sym
** sch_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/OpAmp_Defect/OpAmp.sch
.subckt OpAmp VDD vout vin_p vin_n VSS i_bias en
*.ipin vin_p
*.ipin vin_n
*.iopin VSS
*.iopin VDD
*.opin vout
*.ipin i_bias
*.ipin en
XM1 vout Active_

## Parsing FET devices
```
Fet structure (both n and p):
<inst_name> <drain> <gate> <source> <bulk> <cell_name>

Example:
XM1 vout Active_load_g VDD VDD sky130_fd_pr__pfet_01v8 ...
```



In [ ]:
def get_fet_devices(netlist_contents):

  # Regular expression to extract FET devices (lines starting with XM)
  regex = r'(?P<inst_name>\w+\d+)\s(?P<drain>\w+)\s(?P<gate>\w+)\s(?P<source>\w+)\s(?P<bulk>\w+)\s(?P<cell_name>sky130_fd_pr__[np]fet_01v8)'

  # Find all matching FET device lines
  fet_devices = re.findall(regex, netlist_contents)

  # Display the extracted FET devices
  #for fet in fet_devices:
  #    print("Inst_name", fet[0])
  #    print("Drain", fet[1])
  #    print("Gate", fet[2])
  #    print("Source", fet[3])
  #    print("Bulk", fet[4])
  #    print("Cell_name", fet[5])
  #    print("")
  return fet_devices


## Add Open Defect to SPICE netlist


In [ ]:
def modify_nelist_open_defect(netlist_contents, fet_device_terminals, defect_device, defect_terminal_name, defect_terminal):
#defect_device = 0
#defect_terminal_name = "Drain"
#defect_terminal = 1 # drain

  inst_name = fet_device_terminals
  defect_terminal_netname = fet_device_terminals[defect_terminal]

  # Display the extracted FET devices
  # print("Inst_name", inst_name)
  # print(defect_terminal_name, defect_terminal_netname)

  # Create 2 additional nets to replace original connected to device terminal
  defect_terminal_netname_replaced_1 = "{}1".format(defect_terminal_netname)
  defect_terminal_netname_replaced_2 = "{}2".format(defect_terminal_netname)

  # String of original device terminals
  original_device_terminals = " ".join(fet_device_terminals)

  fet_device_terminals[defect_terminal] = defect_terminal_netname_replaced_1

  # String of modified device terminals
  modified_device_terminals = " ".join(fet_device_terminals)

  print(original_device_terminals)
  print(modified_device_terminals)
  print("")

  # Replace device terminals
  modified_netlist_contents = re.sub(original_device_terminals, modified_device_terminals, netlist_contents)
  # Replace remain occurances of net
  modified_netlist_contents = re.sub(r"\b{}\b".format(defect_terminal_netname), defect_terminal_netname_replaced_2, modified_netlist_contents)
  # Add open defect by adding a resistor
  modified_netlist_contents = re.sub("[\*]{2}.ends", "R_open {} {} 1G m=1 \n**.ends".format(defect_terminal_netname_replaced_1, defect_terminal_netname_replaced_2),
                            modified_netlist_contents, count=1)

  return modified_netlist_contents

In [ ]:
def generate_spice_netlists(netlist_contents, fet_device_terminals, defect_device_num, defect_terminal_name, defect_terminal):

  modified_netlist_contents = modify_nelist_open_defect(netlist_contents, fet_device_terminals, defect_device_num, defect_terminal_name, defect_terminal)
  #print(modified_netlist_contents)

  # Open a file for writing modified netlist
  with open("./generated_netlists/test_analog_{}_{}_{}.spice".format(defect_device_num,fet_device_terminals[0],defect_terminal_name), "w") as file:
      file.write(modified_netlist_contents)

In [ ]:
defect_terminal_name = "Drain"
defect_terminal = 1

fet_devices = get_fet_devices(netlist_contents)

# Create netlists with defects injected
for defect_device_num in range(0,len(fet_devices)-1):
  fet_device_terminals = list(fet_devices[defect_device_num])
  generate_spice_netlists(netlist_contents, fet_device_terminals, defect_device_num, defect_terminal_name, defect_terminal)

XM1 vout Active_load_g VDD VDD sky130_fd_pr__pfet_01v8
XM1 vout1 Active_load_g VDD VDD sky130_fd_pr__pfet_01v8

XM2 Active_load_g Active_load_g VDD VDD sky130_fd_pr__pfet_01v8
XM2 Active_load_g1 Active_load_g VDD VDD sky130_fd_pr__pfet_01v8

XM3 Active_load_g vin_p I_mirror_d VSS sky130_fd_pr__nfet_01v8
XM3 Active_load_g1 vin_p I_mirror_d VSS sky130_fd_pr__nfet_01v8

XM4 vout vin_n I_mirror_d VSS sky130_fd_pr__nfet_01v8
XM4 vout1 vin_n I_mirror_d VSS sky130_fd_pr__nfet_01v8

XM5 I_mirror_d I_mirror_g VSS VSS sky130_fd_pr__nfet_01v8
XM5 I_mirror_d1 I_mirror_g VSS VSS sky130_fd_pr__nfet_01v8

XM6 i_bias I_mirror_g VSS VSS sky130_fd_pr__nfet_01v8
XM6 i_bias1 I_mirror_g VSS VSS sky130_fd_pr__nfet_01v8

XM9 en_n en VDD VDD sky130_fd_pr__pfet_01v8
XM9 en_n1 en VDD VDD sky130_fd_pr__pfet_01v8

XM10 en_n en VSS VSS sky130_fd_pr__nfet_01v8
XM10 en_n1 en VSS VSS sky130_fd_pr__nfet_01v8

XM7 i_bias en I_mirror_g VSS sky130_fd_pr__nfet_01v8
XM7 i_bias1 en I_mirror_g VSS sky130_fd_pr__nfet_01v8

XM

In [ ]:
# Print Modified Netlist
# print(netlist_contents)